In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install peft
!pip install evaluate
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylin

In [12]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.0 MB/s eta 0:00:00


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('Swamitucats/M2M100_Sanskrit_English')
tokenizer = AutoTokenizer.from_pretrained('Swamitucats/M2M100_Sanskrit_English')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.84M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

In [5]:
from peft import LoraConfig, get_peft_model, TaskType
lora_config = LoraConfig(r=16,
                         lora_alpha = 32,
                         target_modules =['v_proj','k_proj','q_proj','fc1','fc2'],
                         lora_dropout = 0.02,
                         bias = 'none',
                         task_type = TaskType.SEQ_2_SEQ_LM)
model_2 = get_peft_model(model, lora_config)
model_2.print_trainable_parameters()

trainable params: 7,471,104 || all params: 491,376,640 || trainable%: 1.5204


In [6]:
len(tokenizer)

128104

In [7]:
#train with first 100000 data only
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/translator/datasets/dataset.csv')
df

,src,tgt
0,दासाश्च यत् स्वप्रभूनां निघ्नाः सर्व्वविषये तु...,"""Exhort servants to be obedient unto their own..."
1,प्राम्प्ट्-मध्ये एवं टङ्कनं कुर्वन्तु : from B...,"""""""At the prompt, type: from Bio dot Seq impor..."
2,गुरुग्रामस्य प्रदीपसङ्गवानः २०१६ तः 'हिमालयस्य...,Gurugram’s PradeepSangwan has been running a ...
3,तेषां स्वप्ना एव अस्माकं प्रेरणा भवेयुः!\n,Their dreams should be our motivation!\n
4,अध्युपयोक्तारं (super user) अथवा मूलोपयोक्तारं...,Change the password to no password or empty pa...
...,...,...
41581,फलितं प्रदर्शितम् ।\n,The output is displayed.\n
41582,तदेत्थं श्रुत्वा तस्य शिष्याणाम् अनेके परस्परम...,"""Many therefore of his disciples, when they ha..."
41583,नगरस्य नाम इन्द्रप्रस्थम् ।\n,The name of city is Indraprastha.\n
41584,"""तयो र्दूतयो र्गतयोः सतो र्योहनि स लोकान् वक्त...","""And when the messengers of John were departed..."


In [8]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files ='/content/drive/MyDrive/translator/datasets/dataset.csv')
raw_datasets

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['src', 'tgt'],
        num_rows: 41586
    })
})

In [9]:
split_datasets = raw_datasets['train'].train_test_split(train_size = 0.92, seed = 20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['src', 'tgt'],
        num_rows: 38259
    })
    test: Dataset({
        features: ['src', 'tgt'],
        num_rows: 3327
    })
})

In [10]:
tokenizer.tgt_lang = 'en'
def tokenized_function(example):
  inputs = [ex for ex in example['src']]
  labels = [ex for ex in example['tgt']]
  model_inputs = tokenizer(inputs, max_length = 256, padding = 'max_length', truncation = True, return_tensors = 'pt', text_target = labels)
  return model_inputs

tokenized_datasets = split_datasets.map(tokenized_function, batched= True)

Map:   0%|          | 0/38259 [00:00<?, ? examples/s]

Map:   0%|          | 0/3327 [00:00<?, ? examples/s]

In [13]:
import evaluate

# Load the metrics
meteor_metric = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")
#bleu_metric = evaluate.load("bleu")
#chrf_metric = evaluate.load("chrf")

def compute_metrics(eval_pred):
    """
    Compute METEOR and BERTScore to evaluate the model during training.

    Args:
    eval_pred (tuple): A tuple containing the predictions and references.

    Returns:
    dict: A dictionary with the computed METEOR and BERTScore.
    """
    predictions, labels = eval_pred

    # Decode the predicted and reference sentences
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute METEOR score
    meteor_results = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_score = meteor_results["meteor"]

    # Compute BERTScore
    b_results = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang="en")

    # Check if b_results["f1"] is a list and compute the mean if needed
    if isinstance(b_results["f1"], list):
        bertscore_f1 = sum(b_results["f1"]) / len(b_results["f1"])
    else:
        bertscore_f1 = b_results["f1"]
    # Compute BLEU score
    #bleu_results = bleu_metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    #bleu_score = bleu_results["bleu"]

    # Compute ChrF++ score
    #chrf_results = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #chrf_score = chrf_results["score"]
    # Return both scores
    return {
        "meteor": meteor_score,
        "bertscore_f1": bertscore_f1,
        #"bleu": bleu_score,
        #"chrf++": chrf_score
    }


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [14]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model_2.to(device)

cuda


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): M2M100ForConditionalGeneration(
      (model): M2M100Model(
        (shared): M2M100ScaledWordEmbedding(128112, 1024, padding_idx=1)
        (encoder): M2M100Encoder(
          (embed_tokens): M2M100ScaledWordEmbedding(128112, 1024, padding_idx=1)
          (embed_positions): M2M100SinusoidalPositionalEmbedding()
          (layers): ModuleList(
            (0-11): 12 x M2M100EncoderLayer(
              (self_attn): M2M100Attention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.02, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_feat

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
checkpoint_path = '/content/drive/MyDrive/translator/results/checkpoint-47821'
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/translator/results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    gradient_accumulation_steps=2,
    fp16=True,
    save_steps = 2500,
    resume_from_checkpoint=checkpoint_path,
    logging_dir='/content/drive/MyDrive/translator/logs',                     # Directory for logs
    load_best_model_at_end=True,

)
trainer = Seq2SeqTrainer(
    model=model_2,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    compute_metrics = compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train(resume_from_checkpoint = checkpoint_path)

Epoch,Training Loss,Validation Loss,Meteor,Bertscore F1,Bleu,Chrf++
10,0.105600,0.112687,0.653703,0.936839,0.361107,55.680316
12,0.103600,0.111932,0.654445,0.936402,0.358650,55.608517


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
